In [1]:
from leakageFuncs import getModel, getTrueGradient
from tensorflow import keras
import tensorflow as tf
from keras import layers
from keras import backend as k
import numpy as np
import pickle as pk
def startTFSession():
    #tf.compat.v1.disable_eager_execution()
    return tf.compat.v1.InteractiveSession
session = startTFSession()

In [2]:
(trainFeatures,trainLabels),(testFeatures,testLabels) =  keras.datasets.mnist.load_data()
trainLabels = keras.utils.to_categorical(trainLabels,10)
testLabels = keras.utils.to_categorical(testLabels,10)
trainFeatures.astype(np.float32)
testFeatures.astype(np.float32)
trainFeatures = np.reshape(trainFeatures,(len(trainFeatures),28,28,1))/255
testFeatures = np.reshape(testFeatures,(len(testFeatures),28,28,1))/255


In [3]:
model = getModel(batchSize=1)

In [4]:
g = getTrueGradient(model,trainFeatures,trainLabels,testFeatures,testLabels,session,stopIter=100,learningRate=1e-2,batchSize=1)

iter {}


KeyboardInterrupt: 